In [1]:
#!pip install transformers
#!pip install tensorflow
import os
import torch
import pandas as pd
import numpy as np
import warnings
from transformers import (
    AutoTokenizer, AutoModelWithLMHead,
    TextDataset,DataCollatorForLanguageModeling,LineByLineTextDataset,
    Trainer, TrainingArguments,
    WEIGHTS_NAME, CONFIG_NAME
)
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)

In [2]:
output_dir="ensamblador/gpt2_espanol_8hx512pos"
model = AutoModelWithLMHead.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device);

In [13]:
bad_words_ids =[ 
    tokenizer.encode('==', add_prefix_space=True), 
    tokenizer.encode('===', add_prefix_space=True),  
    tokenizer.encode('====', add_prefix_space=True),
    tokenizer.encode('<pad>', add_prefix_space=True)]

def test_model(device, model, text):
    input_ids = tokenizer.encode(text, return_tensors='pt').to(device) # getting out output
    #attention_mask = tokenizer.encode(seeds, return_tensors='pt').to(device)
    
    print("generando beams para input: ", text,"\n==================================================")
    beam_output = model.generate(
        input_ids,
        max_length = 150,
        min_length = 50,
        #early_stop=True,
        do_sample=True,
        top_k=100, 
        top_p=0.85,
        num_beams = 10,
        bad_words_ids = bad_words_ids,
        temperature = 0.9,
        length_penalty=3,
        no_repeat_ngram_size=2,
        repetition_penalty=2.0,
        num_return_sequences=10,
        bos_token_id = tokenizer.bos_token_id,
        eos_token_id = tokenizer.eos_token_id,
        pad_token_id =tokenizer.pad_token_id
    )
    
    decoded_beams = []
    
    
    for beam in beam_output:   
        decoded_beams.append({'text':text, 'generado':tokenizer.decode(beam).split('  ')[0]})
        #print ("\n\n",tokenizer.decode(beam))
        
    return decoded_beams

In [14]:
generated = test_model(device, model, "El 27 de enero del año 2010")

generando beams para input:  El 27 de enero del año 2010 


In [15]:
pd.DataFrame.from_records(generated)

,text,generado
0,El 27 de enero del año 2010,"El 27 de enero del año 2010, se anunció que la serie había sido renovada para una segunda temporada. La serie fue estrenada el de febrero de 2011 en Estados Unidos por la cadena The CW. El 13 de mayo de 2012, ABC renovó ''The Young and the Restless'' para un segundo spin-off, titulado ''Glee: The Music, Vol. 2''. El 12 de julio de 2013, NBC renovó los derechos de transmisión para la tercera temporada de ''American Horror Story: Freak Showcase'', que se estrenó el 15 de junio de 2014. En abril de 2015, CBS renovó la primera temporada para otra vez en septiembre de octubre de ese mismo año, pero se emitió el 19 de nuevo el 16 de noviembre de 2018."
1,El 27 de enero del año 2010,"El 27 de enero del año 2010, se anunció que la serie había sido renovada para una segunda temporada. La serie fue estrenada el 13 de julio de 2011 en Estados Unidos por la cadena The CW. El 10 de mayo de 2012, ABC renovó ''The Young and the Restless'' para un segundo spin-off titulado ''Grey's Anatomy''. El 11 de junio de 2013, NBC renovó su serie para otra tercera temporada titulada ''NCIS: Los Ángeles''. En agosto de 2014, CBS ordenó la cuarta temporada de ''Los So Random Events'', que se estrenó el 15 de septiembre de 2015. Más tarde comenzó transmitirse el 18 de octubre de ese mismo año, pero debido al aire el 12 de febrero de 2016. Fue renovada"
2,El 27 de enero del año 2010,"El 27 de enero del año 2010, el papa Benedicto XVI lo nombró obispo auxiliar de la Arquidiócesis de Buenos Aires. Recibió la ordenación episcopal el 19 de febrero del mismo año, en la Basílica de Nuestra Señora de Luján (Buenos Aires). Su consagración episcopal tuvo lugar el 21 de marzo de 2011, manos del Nuncio Apostólico en Argentina, Mons. Juan Pablo II. Tomó posesión oficial de su diócesis el 20 de abril de ese año. El de mayo de 2012 fue nombrado por el Papa Francisco como co-consagruptor titular de las diócesis de Paraná, Santa Fe, San Isidro, Mar del Valle de los partidos bonaerenses de La Plata, Gualeguaychú, Diamante, Tres Arroyos, Chivillarreale, Nueve de Villaguay,"
3,El 27 de enero del año 2010,"El 27 de enero del año 2010, se anunció que la serie había sido renovada para una segunda temporada. La serie fue estrenada el de abril de 2011 en Estados Unidos por la cadena The CW. En Latinoamérica se estrenó el 11 de mayo de 2012 en FOX. El 13 de julio de 2013, ABC ordenó una nueva serie de televisión basada en los acontecimientos de la primera temporada de ''The Walking Dead'' titulada ''American Horror Story: Covenant''. El de septiembre de 2014, AMC emitió un episodio especial titulado ''America's Next Door to Die Hardline'', basado en el 14 de junio de ese año 2011. Fue emitido el 15 de nuevo el 18 de este mismo año, esta vez, pero fue estrenado el 19 de 2015, como ''"
4,El 27 de enero del año 2010,"El 27 de enero del año 2010, el '''cantón de fue suprimido el 22 de marzo de 2015 sus comunas pasaron formar parte; cuatro del nuevo cantón de la fracción de comuna que le daba su nombre se unió con las demás fracciones para que, por medio de una cantonal, fueran creados los nuevos cantones de Saint-Gingolph Chambéry-1. El 31 de diciembre de 2016 pasó ser una comuna delegada la nueva comuna de Val-de-Vaucluse al fusionarse con la comuna vecina de Pont-Ville. Se ubica en el distrito de Villeneuve. Su población de abril de Monthey</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>"
5,El 27 de enero del año 2010,"El 27 de enero del año 2010, la banda anunció través de sus redes sociales que el nuevo álbum sería lanzado el 21 de febrero del mismo año. El 11 de abril de 2010 se lanzó el primer sencillo del álbum, titulado ""Wish You Were Here"". La canción alcanzó el número uno en las listas de Reino Unido, Canadá, Australia, Francia, Alemania, Irlanda, Italia, Países Bajos, Nueva Zelanda, Noruega, Suecia Suiza. En los Estados Unidos, fue certificado plat